In [18]:
from neo4j import GraphDatabase, RoutingControl
import neo4j
from dotenv import load_dotenv
import os
import pandas as pd
# from neomodel import clear_neo4j_database, db
# import neomodel
import json
import csv

In [19]:
load_dotenv()
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(os.getenv("USERNAME"), os.getenv("PASSWORD")))

Create patients

In [20]:
base_path = "/Users/gianluca/github/synthea/output/synthea_"
num = 1000
path = f"{base_path}{num}/csv"
patients_df = pd.read_csv(f"{path}/patients.csv")
print(len(patients_df))
patients_df.to_csv(f"/Users/gianluca/neo4j-enterprise-5.26.12/import/synthea/patients_{num}.csv")


1283


In [21]:

observation_df = pd.read_csv(f"{path}/observations.csv")
cols = ["Body Height","Body mass index (BMI) [Ratio]","Respiratory rate", "Heart rate", "Body Weight"]
file_names = ["bodyHeight", "bmi", "respiratoryRate", "heartRate", "bodyWeight"]
# Build a dictionary: patient_id -> {file1_timestamps, file1_values, ...}
patient_data = {}
for file, value in zip(file_names, cols):
    temp_df = observation_df[['DATE', 'PATIENT', 'DESCRIPTION', 'VALUE']].loc[observation_df['DESCRIPTION'] == value]
    grouped = temp_df.groupby('PATIENT')
    for patient, group in grouped:
        timestamps = group['DATE'].tolist()
        values = [float(v) for v in group['VALUE'].tolist()]
        if patient not in patient_data:
            patient_data[patient] = {}
        patient_data[patient][f'{file}_timestamps'] = json.dumps(timestamps)
        patient_data[patient][f'{file}_values'] = json.dumps(values)
# Prepare headers
headers = ['patient']
for file in file_names:
    headers.append(f'{file}_timestamps')
    headers.append(f'{file}_values')
# Write to a single CSV
with open(f'/Users/gianluca/neo4j-enterprise-5.26.12/import/synthea/timeseries_{num}.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()
    for patient, pdata in patient_data.items():
        row = {'patient': patient}
        for file in file_names:
            row[f'{file}_timestamps'] = pdata.get(f'{file}_timestamps', json.dumps([]))
            row[f'{file}_values'] = pdata.get(f'{file}_values', json.dumps([]))
        writer.writerow(row)
